# 1. Load Libraries

## 1.1 Installation

In [424]:
#import yfinance as yf
#!pip install --upgrade pip
#!pip install psycopg2-binary
#!pip install psycopg2

## 1.2 Load Libraries

In [425]:
import yfinance as yf
from datetime import date
import psycopg2
import json
from pandas_datareader import data as pdr

# 2. Connect to PostgresDB

### docker inspect postgres
### docker run --name postgres -e POSTGRES_PASSWORD=123 -d postgres

In [432]:
cur = None;
conn = None;
def connectToDB(): 
    global cur;
    global conn;
    cred = {"host":"localhost","port":"5432","database":"postgres","user":"postgres","password":"123"}
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(
        host=cred["host"],
        port=cred['port'],
        database=cred["database"],
        user=cred["user"],
        password=cred["password"])

    # create a cursor
    cur = conn.cursor()

connectToDB();
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')
# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


# 3. Downloading Data

## 3.1 Utility Functions

In [427]:
def doesTableExist(tablename):
    checkExistCmd = "select exists(select * from information_schema.tables where table_name='"+tablename+"')"
    cur.execute(checkExistCmd)
    return cur.fetchone()[0]
    
def dropTable(tablename):
    dropCmd = "DROP TABLE "+tablename+";";
    cur.execute(dropCmd)
    conn.commit()

In [428]:
# Dropping Tables (Optional)
# dropTable("daily_prices")
# dropTable("predicted_daily_prices")
# dropTable("tickers")

In [429]:
NEW_INSTALLATION = True
if doesTableExist("tickers"): 
    NEW_INSTALLATION = False

if NEW_INSTALLATION:
    # tickers Table
    createCmd = """ CREATE TABLE tickers (
                t_id SERIAL PRIMARY KEY,
                t_name VARCHAR(5),
                t_full_name VARCHAR(255)
                )
            """ 
    cur.execute(createCmd)
    conn.commit()
    
    # Daily Prices Table
    createCmd = """ CREATE TABLE daily_prices (
                date DATE,
                ticker_id SERIAL,
                open DECIMAL,
                high DECIMAL,
                low DECIMAL,
                close DECIMAL,
                adj_close DECIMAL,
                volume REAL,
                PRIMARY KEY(date,ticker_id),
                CONSTRAINT fk_dp_ticker FOREIGN KEY(ticker_id) REFERENCES tickers(t_id)
                )
            """  
    
    cur.execute(createCmd)
    conn.commit()
    
    # Predict Prices
    createCmd = """ CREATE TABLE predicted_daily_prices (
                date DATE,
                ticker_id SERIAL,
                predicted DECIMAL,
                PRIMARY KEY(date,ticker_id),
                CONSTRAINT fk_pdp_ticker FOREIGN KEY(ticker_id) REFERENCES tickers(t_id)
                )
            """  
    
    cur.execute(createCmd)
    conn.commit()

# 4. Downloading Data (One Time)

## 4.1 DB Utility Functions 

In [430]:
def addTickerToDB(ticker_name,ticker_fullname):
    insertCmd = "INSERT INTO tickers (t_name,t_full_name) VALUES('"+ticker_name+"','"+ticker_fullname+"') RETURNING *";
    cur.execute(insertCmd);
    res = cur.fetchone();
    return not res is None 

def getTickerId(ticker_name):
    selCmd = "SELECT * FROM tickers WHERE t_name = '"+ticker_name+"'";
    cur.execute(selCmd);
    res = cur.fetchone();
    return res[0];

def insertDailyPrices(date,ticker_id,open,low,high,close,adj_close,volume):
    insertCmd = """INSERT INTO daily_prices(date,ticker_id,open,low,high,close,adj_close,volume)
                VALUES('{date}',{ticker_id},{open},{low},{high},
                {close},{adj_close},{volume}) RETURNING *
                """.format(date=date,ticker_id=ticker_id,
                           open=open,high=high,low=low,
                           close=close,adj_close=adj_close,volume=volume);
    cur.execute(insertCmd);
    res = cur.fetchone();
    return not res is None 

## 4.2 Download

In [431]:
def download(tickers,start_date,end_date):
    # enable converting stocks data to data frame
    yf.pdr_override() # <== that's all it takes :-)
    
    # Loop ticker by ticker    
    for ticker in tickers:
        print("------------")
        print("Adding "+ticker+" to Database...")
        addTickerToDB(ticker,ticker) #<-- update with full name
        t_id = getTickerId(ticker)
        print(ticker+" index been added to the database");
        
        # download data for 10 Years!
        data_df = pdr.get_data_yahoo(ticker, start=START_DATE, end=END_DATE)
        # Storing data into Postgres
        print("Downloading Historical Data for {ticker_name}...".format(ticker_name=ticker))
        
        #cur_year = start_date.strftime("%Y")
        #cur_year = datetime.strptime("%Y")
        for datetime, row in data_df.iterrows():
            #year = datetime.strptime(datetime,"%Y");
            #if not year == cur_year:
            #    print("Downloading data for {year}".format(year))
            #    cur_year=year;
            
            date = pd.to_datetime(datetime).date()
            open_price = "{:f}".format(row['Open'])
            high_price = "{:f}".format(row['High'])
            low_price = "{:f}".format(row['Low'])
            close_price = "{:f}".format(row['Close'])
            adj_close_price = "{:f}".format(row['Adj Close'])
            volume = "{:f}".format(row['Volume'])
            insertDailyPrices(date,t_id,open_price,high_price,low_price,close_price,adj_close_price,volume)
        print("{ticker} Historical Daily Prices been Downloaded".format(ticker=ticker))
            
if NEW_INSTALLATION:
    dow30_tickers = ['AAPL','AMGN','AXP','BP','CAT','CRM','CSCO','CVX','DIS','DOW','GS','HD','HON','IBM',
    'INTC','JNJ','JPM','KO','MCD','MMM','MRK','MSFT','NKE','PG','TRV','UNH','V','VZ','WBA','WMT']
    today = date.today()
    download(dow30_tickers,'2013-01-01',today);
    conn.commit()